# Introduktion till Prompt Engineering
Prompt engineering handlar om att utforma och optimera uppmaningar för uppgifter inom naturlig språkbehandling. Det innebär att välja rätt uppmaningar, justera deras parametrar och utvärdera hur de presterar. Prompt engineering är avgörande för att uppnå hög noggrannhet och effektivitet i NLP-modeller. I det här avsnittet kommer vi att gå igenom grunderna i prompt engineering med hjälp av OpenAI-modeller för att utforska området.


### Övning 1: Tokenisering
Utforska tokenisering med tiktoken, en snabb och öppen tokenizer från OpenAI
Se [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) för fler exempel.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Övning 2: Kontrollera inställningen av Github Models-nyckeln

Kör koden nedan för att kontrollera att din Github Models-endpoint är korrekt konfigurerad. Koden testar bara en enkel prompt och verifierar svaret. Inmatningen `oh say can you see` bör kompletteras med något i stil med `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Övning 3: Påhitt

Utforska vad som händer när du ber LLM:en att ge svar på en fråga om ett ämne som kanske inte existerar, eller om ämnen som den kanske inte känner till eftersom de ligger utanför dess förtränade datamängd (till exempel nyare händelser). Se hur svaret förändras om du provar en annan fråga eller en annan modell.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Övning 4: Instruktionsbaserad
Använd variabeln "text" för att ange huvudinnehållet 
och variabeln "prompt" för att ge en instruktion som är relaterad till huvudinnehållet.

Här ber vi modellen att sammanfatta texten för en elev i andra klass


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Övning 5: Komplex prompt
Testa en förfrågan som innehåller system-, användar- och assistentmeddelanden  
Systemet sätter assistentens kontext  
Användar- och assistentmeddelanden ger kontext för en konversation med flera turer

Observera hur assistentens personlighet är inställd på "sarkastisk" i systemkontexten.  
Prova att använda en annan personlighetskontext. Eller testa en annan serie av in- och utgående meddelanden


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Övning: Utforska din intuition
Exemplen ovan ger dig mönster som du kan använda för att skapa nya uppmaningar (enkla, komplexa, instruktioner osv.) – prova att skapa andra övningar för att utforska några av de andra idéerna vi har pratat om, som exempel, ledtrådar och mer.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Vi strävar efter noggrannhet, men var medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
